# Imports

In [1]:
import pandas as pd
#Dict approach
import glob
import os
import sys
from pathlib import Path
#from typing import Any, Optional
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import h5py

# Load genome loaders

In [2]:
#Load genome loader for salk cluster
sys.path.append('/iblm/netapp/home/jjaureguy/genome_loader/genome-loader')

In [2]:
#Load genome loader for runAI cluster
sys.path.append('/home/jovyan/home/jjaureguy/genome_loader/genome-loader')

In [3]:

import genome_loader.write_h5
import genome_loader.encode_data
import genome_loader.get_data
import genome_loader.get_encoded
import genome_loader.load_data
import genome_loader.load_h5

from genome_loader.write_h5 import write_encoded_genome


# Load in dataframes for Train/valid/test on salk cluster


In [4]:
# Total Dataset not split 
#unified_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/unified_bed_df.txt',sep='\t')
#frag_tn5_h5_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/frag_tn5_h5_df.txt',sep='\t')

#OHE for AGCT
one_hot_enc_genome = h5py.File('/iblm/netapp/data4/jjaureguy/out_dir/genome_onehot.h5','r')

# Train
frag_tn5_train_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/train_frag_tn5_h5_df.txt',sep='\t')
train_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/train_peak_df_filtered.txt',sep='\t')

# Validation
frag_tn5_valid_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/valid_frag_tn5_h5_df.txt',sep='\t')
valid_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/valid_peak_df_filtered.txt',sep='\t')
# Test
frag_tn5_test_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/test_frag_tn5_h5_df.txt',sep='\t')
test_bed_file_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/test_peak_df_filtered.txt',sep='\t')




# Load in dataframes for Train/valid/test on runAI cluster


In [4]:
#OHE for AGCT
one_hot_enc_genome = h5py.File('/home/jovyan/data4/jjaureguy/out_dir/genome_onehot.h5','r')
# Train

frag_tn5_train_df = pd.read_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/train_frag_tn5_h5_df_runai.txt',sep='\t')
train_bed_file_df = pd.read_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/train_peak_df_filtered.txt',sep='\t')

# Validation
frag_tn5_valid_df = pd.read_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/valid_frag_tn5_h5_df_runai.txt',sep='\t')
valid_bed_file_df = pd.read_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/valid_peak_df_filtered.txt',sep='\t')

# Test
frag_tn5_test_df = pd.read_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/test_frag_tn5_h5_df_runai.txt',sep='\t')
test_bed_file_df = pd.read_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/test_peak_df_filtered.txt',sep='\t')



# Dataframe

# Dataset Custom Class

In [5]:
# Imports
from enum import Enum
from pathlib import Path
from typing import Any, Optional
from torch import nn
import pytorch_lightning as pl
import h5py
from tqdm.notebook import tqdm
#from .transforms import KmerShuffle

# Tester Code for dataset class experimental code

In [ ]:

# H5 files function from class

# Empty dict for h5 file paths and h5 files tn5 inserttion cuts
h5_files = {}
def get_h5(h5_path):
        # checks if h5 path is in h5 files
        if h5_path in h5_files:
            return h5_files[h5_path]
        else:
            # key, values dictionary
            h5_files[h5_path] = h5py.File(h5_path, "r")

#Function for selecting 1024 bp window of X features(one hot encoded ref genome peak range)
def get_mid_point(start, end):
    np.random.seed(0)
    mid_point = range(start,end)
    mid_point = np.random.choice(mid_point, 1, replace=True)
    #print('mid pt selection random', (mid_point))
    mid_point = mid_point.item()
    #print(mid_point)
    return mid_point

def get_x_window(mid_point):
    #1) pick random point in range_num(list) and set to variable
    #2) Check that the point is within the bounds of +/- 512 of end of OHE chrom boundary 
    #3) Create window of 1024 window around point 
    bounds = range(mid_point-512, mid_point + 512)
    # 
    mid_point = list(bounds)

    return mid_point

def get_y_window(mid_point):
    # same thing but without check since 256 bp window is within the 1024bp window
    mid_point = list(range(mid_point-128, mid_point + 128))
    return mid_point
        
# T = total_tn5_counts_per_chrom
# C = Counts per window(summed 256 bp region)
def normalize_atac_reads(Tn5_counts_window, T):
    counts = []
    #******Figure out how to iterate over the multiple dimension array and calucate the normaliztion....
    for i in range(0,len(Tn5_counts_window)):
        C = sum(Tn5_counts_window[i])
        print('C',C)
        length = len(Tn5_counts_window[i])
        print('length',length)
        norm = (C*10**9/(T*(length)))
        norm = round(norm)
        counts.append(norm)
    return counts
def bin_window(window,  bin_size):
        length_window = len(window)
        bins = []
        for i in range(0, length_window, bin_size):
            bins.append((window[i:i+bin_size]))
        print(bins)
        return bins
    
# Dummy index
index = range(1,10)
print(index)
# bam file index 
for i in index:
    bam_file_index = (index[i] % len(frag_tn5_valid_df))
    # Peak index 
    peak_index = (index[i] // len(frag_tn5_valid_df))
    # Coordinates for unified bed fil 
    coords = valid_bed_file_df.iloc[peak_index] 

    #Older code to revert back to 
    # Start end range
    #start_end = range(coords.start,coords.end)
    #print('start and end', start_end)
    # valid_bed_file_df[coords.start:coords.end]

    window = (get_mid_point(coords.start,coords.end))
    #print(coords.start,coords.end)
    #print(window)



    # X Feature(one hot encoded genome for chromosome number of one hot enc h5py file and start end postions)
    x =  one_hot_enc_genome[coords.chrom]['onehot'][get_x_window(window)]


    counter = 0
    
    # if x.all()==False:
    #     print(x.all())
    #     print('hello')
#     for i in range(len(x)):
#         if x.all() < 1:
#             counter +=1
#     if counter >0:
#         #print('contains Ns')
#         # then we discard and move on 
#        # hd5_tn_test['chr1']['depth'][()]>0
#         continue
    
#     elif counter==0:
#         print('true')
#         print(coords.start,coords.end)
#         print(window)



    # call geth5 to populate h5 dictionary
    get_h5(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('diff_frag_h5_path')])
    get_h5(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('int_gamma_diff_frag_h5_path')])
    get_h5(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('salm_int_gamma_diff_frag_h5_path')])
    get_h5(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('salm_diff_frag_h5_path')])
    #print(h5_files)

    #h5_files
    # we need this to be dynamic, path is hardcoded
    #y = h5_files.get('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments/int_gamma_mb/frag_depths.h5')

    
    h5_tn5_read_count_diff = h5_files.get(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('diff_frag_h5_path')])
    h5_tn5_read_count_int_gamma = h5_files.get(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('int_gamma_diff_frag_h5_path')])
    h5_tn5_read_count_salm_int_gamma = h5_files.get(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('salm_int_gamma_diff_frag_h5_path')])
    h5_tn5_read_count_salm_diff = h5_files.get(frag_tn5_valid_df.iloc[ bam_file_index,frag_tn5_valid_df.columns.get_loc('salm_diff_frag_h5_path')])
    
    y_1  = (h5_tn5_read_count_diff[coords.chrom]['depth'][get_y_window(window)])
    y_1 = bin_window(window=y_1,bin_size=128)
    #print(len(y_1))
    T_1 = h5_tn5_read_count_diff.attrs["total_sum"]
    #print('T 1',T)
    norm_1 = normalize_atac_reads(y_1, T_1)
    print('normal', normal)
    
    y_2 = (h5_tn5_read_count_int_gamma[coords.chrom]['depth'][get_y_window(window)])
    y_2 = bin_window(window=y_2,bin_size=128)
    T_2 = h5_tn5_read_count_int_gamma.attrs["total_sum"]
    norm_2 = normalize_atac_reads(y_2, T_2)
 
    y_3 = (h5_tn5_read_count_salm_int_gamma[coords.chrom]['depth'][get_y_window(window)])
    y_3 = bin_window(window=y_3,bin_size=128)
    T_3 = h5_tn5_read_count_salm_int_gamma.attrs["total_sum"]
    norm_3 = normalize_atac_reads(y_3, T_3)
        
    y_4 = (h5_tn5_read_count_salm_diff[coords.chrom]['depth'][get_y_window(window)])
    y_4 = bin_window(window=y_4,bin_size=128)
    T_4 = h5_tn5_read_count_salm_diff.attrs["total_sum"]
    norm_4 = normalize_atac_reads(y_4, T_4)
    
    #print(type(y_1))
    #y = np.concatenate((y_1,y_2,y_3,y_4))
    norm_total = np.stack((norm_1,norm_2,norm_3,norm_4))
    # print('len x', len(x))
    # print('len y', len(y))
    #print(x.shape,y_1.shape,y_2.shape,y_3.shape,y_4.shape)
    #print(x.shape, y.shape)
    
    print(len(x))
    print(type(x))
    print(x.shape)
    print(x.ndim)
    # if np.any(x>0):
    #     print('true')
    # else:
    #     print('false')
    # counter = 0
    # for i in np.nditer((x)):
    #     print(i)
    #     #x.all(i):
#     v = np.array([0,0,0,0])
#     print(np.any(np.all(np.isin(x,v,True),axis=1)))
    
#     # Get a flattened 1D view of 2D numpy array
#     flatten_arr = np.ravel(x)
    # Check if all value in 2D array are equal
    if ([0,0,0,0] == x).all(axis = 1).any():
        print('true')
    else: 
        print(x)
        print('false')
    
    



# DataSet Class

In [58]:
import h5py 
import random
from collections import defaultdict

class Dataset_class(Dataset):
    """Dataset_class Dataset.

    Parameters
    ----------
    one_hot_enc_genome object: pass the H5py file object. One-hot encoded genome `.h5` file. e.g. from genome-loader.
    unified_bed_file_df : pass the df object. DF of bed file containing unified peak set of (1st 5 patients). Chrom, start, end.  
    frag_tn5_h5_df: pass the df object. DF containing H5py paths for Tn5 read counts. Patient ID, treatment type, H5py path column containg Tn5 read counts
    """
    # class constructor(initialize ohe, bed file, frag df file)
    def __init__(self, one_hot_enc_genome, unified_bed_file_df, frag_tn5_h5_df,bin_size, dtype = torch.float32):
        super().__init__()
        self.bin_size = bin_size
        self.one_hot_enc_genome = one_hot_enc_genome
        self.unified_bed_file_df = unified_bed_file_df
        self.frag_tn5_h5_df_1 = frag_tn5_h5_df
        self.frag_tn5_h5_df_2 = frag_tn5_h5_df
        self.frag_tn5_h5_df_3 = frag_tn5_h5_df
        self.frag_tn5_h5_df_4 = frag_tn5_h5_df
        self.dtype = dtype
        # Empty dictionary for h5 file paths
        self.h5_files = {}
        self.d = defaultdict(list)

    # Takes in h5 path from Tn5 from frag_tn5_h5_df column containing h5 paths of Tn5 counts
    def get_h5(self, h5_path):
        # checks if h5 path is already in dictionary of h5py file paths
        if h5_path in self.h5_files:
            # returns them if there already there
            return self.h5_files[h5_path]
        else:
            # Sets the path as a key, and the value as the read file
            self.h5_files[h5_path] = h5py.File(h5_path, "r")
            
    """Function: selects random midpoint from X features(one hot encoded ref genome peak range)
          1) pick random point in range_num(list) and set to variable
    """
    def get_mid_point(self, start, end):
        #np.random.seed(0)
        mid_point = int((start+end)/2)
        #mid_point = np.random.choice(mid_point, 1, replace=True)
        #print('mid pt selection random', (mid_point))
        #mid_point = mid_point.item()
        return mid_point
    
    
    """Function: selects 1024 bp window of X features(one hot encoded ref genome peak range)
          1) utilizes mid point from get_mid_point
          2) creates a 1024 window size around midpoint
    """
    def get_x_window(self,mid_point):
        bounds = (mid_point-512, mid_point + 512)
        self.d['1024_bp_start_end'] = bounds
        bounds = range(mid_point-512, mid_point + 512)
        mid_point = list(bounds)
        return mid_point
    
    """Function: selects 256 bp window of Y features(Tn5 frag counts from h5py file)
          1) utilize sames mid point from get_mid_point
          2) creates a 256 window size around midpoint
    """
    def get_y_window(self,mid_point):
        # same thing but without check since 256 bp window is within the 1024bp window
        mid_point = list(range(mid_point-128, mid_point + 128))
        return mid_point
    
    
    """Function: Takes Total Tn5 counts per chromosome and counts per window
          Calculates the Atac normalization of Tn5 cut sites counts
          Tn5_counts_window = Tn5 bp window that corresponds to the OHE region of the genome (X feature)
          C = sum of the Counts per window(summed bp region)
          T = total_tn5_counts_per_chrom for h5py file
    """
    def normalize_atac_reads(self, Tn5_counts_window, T,d):
        self.Tn5_counts_window = Tn5_counts_window
        self.T = T
        counts = []
        
        for i in range(len(Tn5_counts_window)):
            C = np.sum(Tn5_counts_window[i])
            self.d['C'].append(C) 
            length = len(Tn5_counts_window[i])
            norm = (C*10**9/(T*(length)))
            counts.append(norm)
        return counts
    
    """Function: Window size and bins by bin_size
          window = 256 bp window
          bin_size = 1,2,4,6,8,16,32,64,128,256
    """
    def bin_window(self, window,  bin_size):
        self.window = window
        self.bin_size = bin_size 
        length_window = len(window)
        bins = []
        for i in range(0, length_window, bin_size):
            bins.append((window[i:i+bin_size]))
        return bins
    """Function: returns length of dataset of X*Y
    """
    def __len__(self):
        return (len(self.unified_bed_file_df)*len(self.frag_tn5_h5_df_1))
    
    """Function: returns X,Y from OHE genome h5py file and Tn5 frag counts h5py file
    
    """
    def __getitem__(self,index: int):
        #Clears C list every iteration
        self.d['C'].clear()
        self.d['y'].clear()
        #create sequence of strings for keys
        dict_keys = ('Chrom','1024_bp_start_end','Patient_ID','y','C')
        #create the dictionary, `my_dictionary`, using the fromkeys() method
        self.d.fromkeys(dict_keys)
        #print('index',index)
        bam_file_index = (index % len(self.frag_tn5_h5_df_1))
        peak_index = (index // len(self.frag_tn5_h5_df_1))
        # object that contains chrom, start, end
        coords = self.unified_bed_file_df.iloc[peak_index]
        # Picks midpoint from coors(bed file peak region)
        
        # Add chrom coords to dictionary 
        self.d['Chrom'] = coords.chrom
        
        window = (self.get_mid_point(coords.start,coords.end))
        # create X from OHE genome at the bed file peak region with 1024 bp window size
        # Window for x and dictionary
        temp = self.get_x_window(window)
        x =  self.one_hot_enc_genome[coords.chrom]['onehot'][temp]
        # Add start,end window coords to dictionary
        #self.d['1024_bp_start_end'] = temp[::len(temp)-1]
        

        # To tensor when running on runAI
        x = torch.as_tensor(x,  dtype=self.dtype)
        # reads in h5py path from Col 1(treatment type) in df of h5py Tn5 insertion cutsite h5 paths
        self.get_h5(self.frag_tn5_h5_df_1.iloc[ bam_file_index, self.frag_tn5_h5_df_1.columns.get_loc('diff_frag_h5_path')])
        
        # Add patient id to dictionary
        self.d['Patient_ID'] = (self.frag_tn5_h5_df_1.iloc[ bam_file_index, self.frag_tn5_h5_df_1.columns.get_loc('patient_id')])
        #Gets the h5py path at the bam index from dictionary of h5py paths
        h5_tn5_read_count_diff = self.h5_files.get(self.frag_tn5_h5_df_1.iloc[ bam_file_index, self.frag_tn5_h5_df_1.columns.get_loc('diff_frag_h5_path')])
        
        
        # Total total_tn5_counts for all chroms
        T_1 = h5_tn5_read_count_diff.attrs["total_sum"]
        #Adds T_1 Tn5 counts value to dictionary
        #self.d['T1_diff_counts'] = T_1
        
        # Sets y1 to 256 bp window from h5py Tn5 insertion cutsites corresponding to OHE genome window
        y_1  = (h5_tn5_read_count_diff[coords.chrom]['depth'][self.get_y_window(window)])
        # Bin window based off bin_size
        y_1 = self.bin_window(window=y_1,bin_size=self.bin_size)
        #Normalize each bin windowy_1 = self.bin_window(window=y_1,bin_size=self.bin_size)
        norm_1 = self.normalize_atac_reads(y_1, T_1, self.d)
        #self.d['Norm_1'] = norm_1

        # reads in h5py path from Col 2(treatment type) in df of h5py Tn5 insertion cutsite h5 paths
        self.get_h5(self.frag_tn5_h5_df_2.iloc[ bam_file_index, self.frag_tn5_h5_df_2.columns.get_loc('int_gamma_diff_frag_h5_path')])
        
        #Gets the h5py path at the bam index from dictionary of h5py paths
        h5_tn5_read_count_int_gamma = self.h5_files.get(self.frag_tn5_h5_df_2.iloc[ bam_file_index, self.frag_tn5_h5_df_2.columns.get_loc('int_gamma_diff_frag_h5_path')])
        # Total total_tn5_counts for all chroms
        T_2 = h5_tn5_read_count_int_gamma.attrs["total_sum"]
        #Adds T_2 Tn5 counts value to dictionary
        #self.d['T2_int_gamma_diff_counts'] = T_2
        # Sets y2 to 256 bp window from h5py Tn5 insertion cutsites corresponding to OHE genome window
        y_2 = (h5_tn5_read_count_int_gamma[coords.chrom]['depth'][self.get_y_window(window)])
        # Bin window based off bin_size
        y_2 = self.bin_window(window=y_2,bin_size=self.bin_size)
        #Normalize bins in window
        norm_2 = self.normalize_atac_reads(y_2, T_2, self.d)
        #self.d['Norm_2'] = norm_2
        # reads in h5py path from Col 3(treatment type) in df of h5py Tn5 insertion cutsite h5 paths
        self.get_h5(self.frag_tn5_h5_df_3.iloc[ bam_file_index, self.frag_tn5_h5_df_3.columns.get_loc('salm_int_gamma_diff_frag_h5_path')])
        #Gets the h5py path at the bam index from dictionary of h5py paths
        h5_tn5_read_count_salm_int_gamma = self.h5_files.get(self.frag_tn5_h5_df_3.iloc[ bam_file_index, self.frag_tn5_h5_df_3.columns.get_loc('salm_int_gamma_diff_frag_h5_path')])
        # Total total_tn5_counts for all chroms
        T_3 = h5_tn5_read_count_salm_int_gamma.attrs["total_sum"]
        #Adds T_3 Tn5 counts value to dictionary
        #self.d['T3_salm_int_gamma_counts'] = T_3
        
        
        # Sets y3 to 256 bp window from h5py Tn5 insertion cutsites corresponding to OHE genome window
        y_3 = (h5_tn5_read_count_salm_int_gamma[coords.chrom]['depth'][self.get_y_window(window)])
        # Bin window based off bin_size
        y_3 = self.bin_window(window=y_3,bin_size=self.bin_size)
        #Normalize bins in window
        norm_3 = self.normalize_atac_reads(y_3, T_3, self.d)
        #self.d['Norm_3'] = norm_3

        # reads in h5py path from Col 4(treatment type) in df of h5py Tn5 insertion cutsite h5 paths
        self.get_h5(self.frag_tn5_h5_df_4.iloc[ bam_file_index, self.frag_tn5_h5_df_4.columns.get_loc('salm_diff_frag_h5_path')])
        #Gets the h5py path at the bam index from dictionary of h5py paths
        h5_tn5_read_count_salm_diff = self.h5_files.get(self.frag_tn5_h5_df_4.iloc[ bam_file_index, self.frag_tn5_h5_df_4.columns.get_loc('salm_diff_frag_h5_path')])
        T_4 = h5_tn5_read_count_salm_diff.attrs["total_sum"]
        
        #Adds T_4 Tn5 counts value to dictionary
        #self.d['T4_salm_counts'] = T_4
        
        # Sets y4 to 256 bp window from h5py Tn5 insertion cutsites corresponding to OHE genome window
        y_4 = (h5_tn5_read_count_salm_diff[coords.chrom]['depth'][self.get_y_window(window)])
        # Bin window based off bin_size
        y_4 = self.bin_window(window=y_4,bin_size=self.bin_size)
        #Normalize bins in window
        norm_4 = self.normalize_atac_reads(y_4, T_4, self.d)
        #self.d['Norm_4'] = norm_4
        # concatenate 4 windows(based off bin_size) to create concatenated bp window of four treatments corresponding to OHE genome 1024 bp window
        y = np.concatenate((norm_1,norm_2,norm_3,norm_4))
        self.d['y'].append(y) 
        # To tensor when running on runAI
        y = torch.as_tensor(y, dtype=self.dtype)

        #Returns tuple of X,y ((1024,4),(bin size related)) size
        return x, y, self.d


In [59]:
# Experimental plots

In [60]:
# Test index of no N's index

DS = Dataset_class(one_hot_enc_genome, train_bed_file_df, frag_tn5_train_df,bin_size = 256)
#type(DS[0])
#len(DS)
DS.__getitem__(0)
#DS.__len__()

(tensor([[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         ...,
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.]]),
 tensor([0.9952, 0.4280, 0.1627, 0.2674]),
 defaultdict(list,
             {'C': [12, 6, 2, 4],
              'y': [array([0.99522483, 0.42800735, 0.16273491, 0.26738884])],
              'Chrom': 'chr1',
              '1024_bp_start_end': (180367, 181391),
              'Patient_ID': 'HPSI0114i-oevr_3'}))

In [48]:
import torch.utils.data as data_utils
import matplotlib.pyplot as plt

#np.random.choice(5, 3, replace=False)
# DS.__len__()
#indices = np.arange(0,2)

indices = np.random.choice(DS.__len__(), 10, replace=False)
#np.random.choice(5, 3, )
subset_10 = data_utils.Subset(DS, indices)

In [49]:
type(subset_10)

torch.utils.data.dataset.Subset

In [79]:
df = pd.DataFrame()
for i in (subset_10):
    #print(i)
    print(type(i[2]['C']))
    for k in i[2]['C']:
            print(k)
    #df = pd.DataFrame([[k] + v[0] for k, v in i[2].items()], 
                   #columns=['id', 'score', 'category'])
        #print((data_list))
        #print('yo')
        #data = pd.Series(data_list)
        #print(data)
        #dict_new = dict(data_list)
        #df = pd.concat(data)
print(df)

<class 'list'>
3
7
7
2
<class 'list'>
2
28
13
10
<class 'list'>
22
21
26
41
<class 'list'>
3
2
4
2
<class 'list'>
70
33
37
102
<class 'list'>
4
1
7
7
<class 'list'>
1
1
70
40
<class 'list'>
38
7
19
58
<class 'list'>
1
5
3
15
<class 'list'>
137
233
86
380
Empty DataFrame
Columns: []
Index: []


In [ ]:
C_list = []
for i in subset_10:
    #print(i)
    for data_list in i[2]['C']:
        C_list.append(data_list)  # Unpack
        #print(x) # Not handled in example
    plt.xlabel('C = count values summed during normalization for 256 bin size')
    # naming the y axis
    plt.ylabel('10,000 Random indexes out of total 7,213,673 indexes from Training dataset')
    # giving a title to my graph
    plt.title('10,000 random data points for C from Training dataset')
    plt.hist((C_list),width=20, color='g')
    # naming the x axis
    C_list.clear()
        #plt.plot(dates, values, label=key)
    #plt.legend()

In [ ]:

y_list = []

for i in subset_10:
    #print(i)
    for data_list in i[2]:
        #print(data_list)
        y_list.append(data_list)  # Unpack
        #print(x) # Not handled in example
    plt.xlabel('Normalized count values summed during normalization for 256 bin size')
    # naming the y axis
    plt.ylabel('10,000 Random indexes out of total 7,213,673 indexes from Training dataset')
    # giving a title to my graph
    plt.title('10,000 random data points for Normalized counts from Training dataset')
    plt.hist((y_list),width=5, color='g')
    # naming the x axis
    y_list.clear()
        #plt.plot(dates, values, label=key)
    #plt.legend()

# Function for graphing X(OHE Genome for 1024 bp window) 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import modisco
import sys
import os
import modisco.visualization
from modisco.visualization import viz_sequence


viz_sequence.plot_weights(DS[6][0], subticks_frequency=20)

# Function for graphing Y(Tn5 insertion cut counts in 256 bp window) 

In [ ]:
import torch
import torchplot as plt

# Takes in y tensor index from dataset class tuple in DS[#][1] format. First index value is the total index for the tuple. Second index is 0 for X and 1 for Y for 2nd parameter. 
def graph_y(y_tensor_index):
    #Sets x axis variable for graph
    x = len(y_tensor_index)
    # Sets Y axis variable for graph
    y = y_tensor_index
    # Plots with torch.arrange since we have a tensor object.
    plt.bar(torch.arange(x),y, width = 1)
    plt.xlabel(f"{x} BP window length, bin_size={DS.bin_size}")
    plt.ylabel("No.of Tn5 read insertion cut sites counts")
    plt.show()

In [ ]:
# Call graph function
graph_y(y_tensor_index=DS[6][1])

# Imports for pytorch lightning and wandb

In [7]:
import wandb
from pytorch_lightning.loggers import WandbLogger
# Pytorch modules
import torch
from torch.nn import functional as F
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
import pytorch_lightning as pl



# Definining our model(Naive CC)

In [8]:
class LitNN(LightningModule):
    def __init__(self,seq_len=1024, channels= 4,num_classes=16, n_layer_1=128, n_layer_2=64, n_layer_3=32 ,lr=1e-3):
        """
        init convolution and activation layers
        Args:
        x: (Nx1x2004)
        class: 

        """
        super().__init__() 
        
        self.conv1 = torch.nn.Conv1d(channels, n_layer_1, kernel_size=3)
        self.batch1 = nn.BatchNorm1d(n_layer_1)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv1d(n_layer_1, n_layer_2, kernel_size=3)
        self.batch2 = nn.BatchNorm1d(n_layer_2)
        self.conv3 = torch.nn.Conv1d(n_layer_2, n_layer_3, kernel_size=3)
        self.batch3 = nn.BatchNorm1d( n_layer_3)
        self.pool = torch.nn.MaxPool1d(4)
        self.fc1 = torch.nn.Linear(480, num_classes)
        
        # optimizer parameters
        self.lr = lr

        # metrics
        self.R2score = torchmetrics.R2Score(num_outputs = 16)

        # optional - save hyper-parameters to self.hparams
        # they will also be automatically logged as config parameters in W&B
        self.save_hyperparameters()

    def forward(self, x):
        """
        forward function describes how input tensor is transformed to output tensor
        Args:
            
        """
        
        batch_size,  seq_len, channels = x.size()
        # Changes the order to batch_size, channels, seq_len 
        x = x.permute(0,2,1)
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.batch3(x)
        x = self.relu(x)
        x = self.pool(x)
        #Flatten layer for fully connected layer
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x
    

    def training_step(self, batch, batch_idx):
        '''needs to return a loss from a single batch'''
        x, y = batch
        logits = self(x)
        loss = F.mse_loss(logits, y)

        # Log training loss
        self.log('train_loss', loss)

        # Log metrics
        self.log('train_R2', self.R2score(logits, y))

        return loss

    def validation_step(self, batch, batch_idx):
        '''used for logging metrics'''
        x, y = batch
        logits = self(x)
        loss = F.mse_loss(logits, y)
        
        # Log validation loss (will be automatically averaged over an epoch)
        self.log('valid_loss', loss)

        # Log metrics
        self.log('valid_R2', self.R2score(logits,  y))
        return loss

    def test_step(self, batch, batch_idx):
        '''used for logging metrics'''
        x, y = batch
        logits = self(x)
        loss = F.mse_loss(logits, y)

        # Log test loss
        self.log('test_loss', loss)

        # Log metrics
        self.log('test_R2', self.R2score(logits, y))
        return loss
    def predict_step(self, batch, batch_idx):
        x = batch
        pred = self(x)
        return pred

    def configure_optimizers(self):
        '''defines model optimizer'''
        return Adam(self.parameters(), lr=self.lr)

# Calculate linear layer

In [ ]:
from torchinfo import summary

model = LitNN()
batch_size = 64
summary(model, input_size=(batch_size, 1024,4))

# Old lightning Mnist code

In [ ]:
class LitNN(LightningModule):

    def __init__(self, seq_len=1024, seq_len_tn5=1024,channels= 4,num_classes=1024, n_layer_1=1280, n_layer_2=640, lr=1e-3):
        '''method used to define our model parameters'''
        super().__init__()
    
        # Dimension of X is (1024,4). Window of OHE genome bp x 5(OHE ACGTN)
        self.layer_1 = torch.nn.Conv1d(seq_len * channels, n_layer_1)
        self.layer_2 = torch.nn.Conv1d(n_layer_1, n_layer_2)
        self.layer_3 = torch.nn.Conv1d(n_layer_2, num_classes)

        # optimizer parameters
        self.lr = lr

        # metrics
        self.R2score = torchmetrics.R2Score(num_outputs = 1024)

        # optional - save hyper-parameters to self.hparams
        # they will also be automatically logged as config parameters in W&B
        self.save_hyperparameters()

    def forward(self, x):
        '''method used for inference input -> output'''
        #print('x dim pre view',x.size())
        batch_size, seq_len, channels = x.size()
        # (b, 1024, 4) -> (b, 1024*4)
        #print('x dim post forward',x.size())
        x = x.view(batch_size, -1)
        #print('x dim post view',x.size())
        x = self.layer_1(x)
        #print('x dim post layer 1',x.size())

        x = F.relu(x)
        #print('x dim post relu 1',x.size())

        x = self.layer_2(x)
        #print('x dim post layer 2',x.size())
        x = F.relu(x)
        #print('x dim post relu 2',x.size())
        x = self.layer_3(x)
        #print('nn x post layer 3', x.size())
        x = F.relu(x)
        #print('returned nn x post relu 3', x.size())
        return x

    def training_step(self, batch, batch_idx):
        '''needs to return a loss from a single batch'''
        x, y = batch
        #print('x dim in trstep',x.size())
        #print('y dim in trstep',y.size())


        logits = self(x)
        loss = F.mse_loss(logits, y)

        # Log training loss
        self.log('train_loss', loss)

        # Log metrics
        self.log('train_R2', self.R2score(logits, y))

        return loss

    def validation_step(self, batch, batch_idx):
        '''used for logging metrics'''
        x, y = batch
        #print('x dim in vstep',x.size())
        #print('y dim in vstep',y.size())

        logits = self(x)
        #print('logits',logits.size())
        #print('y in vald step',y.size())
        loss = F.mse_loss(logits, y)
        #print('loss',loss)
        # Log validation loss (will be automatically averaged over an epoch)
        self.log('valid_loss', loss)

        # Log metrics
        self.log('valid_R2', self.R2score(logits,  y))
        return loss

    def test_step(self, batch, batch_idx):
        '''used for logging metrics'''
        x, y = batch
        #print('x dim in tstep',x.size())
        #print('y dim in tstep',y.size())

        logits = self(x)
        loss = F.mse_loss(logits, y)

        # Log test loss
        self.log('test_loss', loss)

        # Log metrics
        self.log('test_R2', self.R2score(logits, y))
        return loss
    def predict_step(self, batch, batch_idx):
        x = batch
        pred = self(x)
        return pred

    def configure_optimizers(self):
        '''defines model optimizer'''
        return Adam(self.parameters(), lr=self.lr)

# Data Loader functions(train, valid, test)

# Define collate function


In [ ]:
from torch.utils.data.sampler import WeightedRandomSampler
import torch

class ATACDataModule(pl.LightningDataModule):
    def __init__(self, one_hot_enc_genome, train_bed_file_df,valid_bed_file_df,test_bed_file_df,frag_tn5_train_df,frag_tn5_valid_df,frag_tn5_test_df, batch_size=64):
        super().__init__()
        self.one_hot_enc_genome = one_hot_enc_genome
        self.train_bed_file_df = train_bed_file_df
        self.valid_bed_file_df = valid_bed_file_df
        self.test_bed_file_df = test_bed_file_df
        self.frag_tn5_train_df = frag_tn5_train_df
        self.frag_tn5_valid_df = frag_tn5_valid_df
        self.frag_tn5_test_df = frag_tn5_test_df
        self.batch_size = batch_size
    # collate function for removing none from batch
    # def collate_fn(self, batch):
    #     batch = list(filter(lambda x: x is not None, batch))
    #     return torch.utils.data.dataloader.default_collate(batch)
    def setup(self, stage=None):
        '''called on each GPU separately - stage defines if we are at fit or test step'''
        # we set up only relevant datasets when stage is specified (automatically set by Pytorch-Lightning)
        if stage == 'fit' or stage is None:
            self.train_ds = Dataset_class(one_hot_enc_genome, train_bed_file_df, frag_tn5_train_df, dtype = torch.float32, bin_size = 64)
            self.valid_ds = Dataset_class(one_hot_enc_genome, valid_bed_file_df, frag_tn5_valid_df,dtype = torch.float32, bin_size = 64)
        if stage == 'test' or stage is None:
            self.test_ds = Dataset_class(one_hot_enc_genome, test_bed_file_df, frag_tn5_test_df,bin_size = 64)
            
    def train_dataloader(self):
        '''returns training dataloader'''
        train_dl = DataLoader(self.train_ds, shuffle=True, batch_size=self.batch_size, num_workers=16)
        return train_dl

    def val_dataloader(self):
        '''returns validation dataloader'''
        valid_dl = DataLoader(self.valid_ds,shuffle=False, batch_size=self.batch_size,num_workers=16)
        return valid_dl

    def test_dataloader(self):
        '''returns test dataloader'''
        test_dl = DataLoader(self.test_ds,  shuffle=False,  batch_size=self.batch_size,num_workers=16)
        # collate_fn=self.collate_fn
        # , num_workers=4
        return test_dl

# WandB

In [ ]:
wandb.login()

In [ ]:
wandb_logger = WandbLogger()

# Training the model

In [ ]:
import torch
import torchmetrics

# Setup  model
model = LitNN(seq_len=1024, channels= 4,num_classes=16, n_layer_1=128, n_layer_2=64, n_layer_3=32,lr=1e-3)


In [ ]:
# Instantiate ATAC Data module
atac = ATACDataModule(one_hot_enc_genome, train_bed_file_df,valid_bed_file_df,test_bed_file_df,frag_tn5_train_df,frag_tn5_valid_df,frag_tn5_test_df, batch_size=64)

In [ ]:
# Trainer initializer
trainer = pl.Trainer(
    gradient_clip_val=100000,
    logger=wandb_logger,
    #overfit_batches= 0.1,
    # fast_dev_run = True,
    accelerator='auto',
    auto_lr_find=True, # W&B integration 
    #devices=1 if torch.cuda.is_available() else None,
    max_epochs=3 # number of epochs
    )

In [ ]:
# Choose best LR automatically
trainer.tune(model,atac)

In [ ]:
lr_finder = trainer.tuner.lr_find(model, atac)

In [ ]:
fig = lr_finder.plot(suggest=True)
fig.show()

In [ ]:
# Define the LR with the best chosen LR
model.lr = 1e-7

In [ ]:
model.lr

In [ ]:
# Train the model
trainer.fit(model, atac)

In [ ]:
# Test model
trainer.test(model, atac)


In [ ]:
# clos wandb
wandb.finish()

# Function for testing H5Py files depth whether read counts existed(QC)

In [ ]:
counter = 0

for i in range(len(hd5_tn_test['chr1']['depth'])):
    if hd5_tn_test['chr1']['depth'][i] > 0:
        counter +=1


   # hd5_tn_test['chr1']['depth'][()]>0

print('The length of data with a for loop: {}'.format((counter)))



In [ ]:
df.loc[0,'read_depth'] = Path(df.loc[0]['bam paths']).parent / "read_depths.h5"

In [ ]:
counter = 0

for i in ((ds)):
        if ds[i] != '[0 0 0 0]':
            counter += 0 

   # hd5_tn_test['chr1']['depth'][()]>0

print('The length of data with a for loop: {}'.format((counter)))


# Check h5 file structure(testing)

In [ ]:
hd5_tn_test = h5py.File('/home/jovyan/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments/diff_mb/frag_depths.h5','r')

In [ ]:
hd5_tn_test

#hd5_tn_test['chrome']['depth']

In [ ]:
type(hd5_tn_test)

In [ ]:
(one_hot_enc_genome.keys())

In [ ]:
one_hot_enc_genome.keys()

In [ ]:
dset = one_hot_enc_genome['chr1']

In [ ]:
dset.keys()